<a href="https://colab.research.google.com/github/shweta-aiml/100daysofcode/blob/master/HindiTextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install indic-transliteration 

     |████████████████████████████████| 102kB 2.0MB/s 
     |████████████████████████████████| 911kB 6.2MB/s 
  Created wheel for splinter: filename=splinter-0.13.0-cp36-none-any.whl size=33304 sha256=0ec0af8d13815ca4a0f0d5c3e366dee433add495725bd5d09affd576a0845ede
  Stored in directory: /root/.cache/pip/wheels/fd/ff/26/2fc56897bcbe58908bbb2002c53affb516e5e6daa425aefdef
Successfully built splinter


In [2]:
import urllib.request
import os
import zipfile
import glob
import numpy as np
import random
from keras import models
from keras import layers
from keras import callbacks
from keras.utils import plot_model
import matplotlib.pyplot as plt
import progressbar
from indic_transliteration import sanscript

import tensorflow as tf

import numpy as np
import os
import time


Using TensorFlow backend.


In [0]:
#load the data , check to see if we can load data from one file lets say premchandra 
#create input and putput sequence of data
#creathe the model 
#train
#predict 


In [0]:
# Path where the corpus will end up.
corpus_path = "corpus"
file_contents = []
# Hyperparameters.
transliteration = True # Transliterates the corpus.
input_length = 40 # Length of the input sequence.
output_length = 1 # Length of the output sequence.
data_set_size = 100000 # Size of the data-set to train on.
num_epochs = 50 # Number of epochs to train.
batch_size = 512 # Batch size during training.
hidden_size = 350 # Size of the hidden layer.
generation_length = 160 # Size of the strings that are generated.

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
#downloading corpus from the github sites
def ensure_corpus():
    """ Makes sure that the corpus is on the hard-drive."""

    # Do nothing if the filder already exists-
    if os.path.exists("corpus") == False:
        # Download the whole git-repository as a zip.
        print("Downloading corpus...")
        corpus_url = "https://github.com/cltk/hindi_text_ltrc/archive/master.zip"
        corpus_zip_path = "master.zip"
        urllib.request.urlretrieve(corpus_url, corpus_zip_path)

        # Unzip the whole git-repository to the corpus-path.
        print("Unzipping corpus...")
        zip_file = zipfile.ZipFile(corpus_zip_path, 'r')
        zip_file.extractall(corpus_path)
        zip_file.close()

        # Remove the zip-file.
        os.remove(corpus_zip_path)

In [0]:
def read_file():
    # Get paths to all files.
    glob_path = os.path.join(corpus_path, "**/*.txt")
    paths = glob.glob(glob_path, recursive=True)
    print("Display all text files ")
    print(paths)
    return paths

def load_file(paths):
    print("Loading all files...")
    
    for path in paths:
        file_content = open(path, "r").read()
        if transliteration == True:
            file_content = sanscript.transliterate(file_content, sanscript.DEVANAGARI, sanscript.IAST)
        file_content = clean_text(file_content)
        file_contents.append(file_content)


In [0]:
#unique list of characters
def get_character_set(text):
    """ Retrieves the unique set of characters. """
    vocab =  sorted(list(set(text)))
    print ('{} unique characters'.format(len(vocab)))
    return vocab


In [0]:
def clean_text(text):
    """ Cleans a text. """

    text = text.replace("\t", " ")
    text = text.replace("\n", " ")
    text = text.replace("।", " ")
    text = text.replace("0", " ")
    text = text.replace("1", " ")
    text = text.replace("2", " ")
    text = text.replace("3", " ")
    text = text.replace("4", " ")
    text = text.replace("5", " ")
    text = text.replace("6", " ")
    text = text.replace("7", " ")
    text = text.replace("8", " ")
    text = text.replace("9", " ")
    text = " ".join(text.split())
    return text

In [0]:
def vectorize_text(vocab,text):
  #vectorizing the text
  # Creating a mapping from unique characters to indices
  char2idx = {u:i for i, u in enumerate(vocab)}
  idx2char = np.array(vocab)

  text_as_int = np.array([char2idx[c] for c in text])
  print('{')
  for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
  print('  ...\n}')
  # Show how the first 13 characters from the text are mapped to integers
  print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))
  return text_as_int,idx2char

In [0]:
# The maximum length sentence we want for a single input in characters
def char_to_seq(text,text_as_int):
  seq_length = 100
  examples_per_epoch = len(text)//(seq_length+1)

  # Create training examples / targets
  char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

  for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

In [10]:
ensure_corpus()
paths = read_file()
load_file(paths)
 # Getting character set.
print("Getting character set...")
global full_text
full_text = " ".join(file_contents)
global character_set
character_set = get_character_set(full_text)
print("Character set:", character_set, len(character_set))
text_int,idx2char = vectorize_text(character_set,full_text)

Unzipping corpus...
Display all text files 
['corpus/hindi_text_ltrc-master/Rahima/main.txt', 'corpus/hindi_text_ltrc-master/Kabeera/main.txt', 'corpus/hindi_text_ltrc-master/miscellaneous/Uddhav/main.txt', 'corpus/hindi_text_ltrc-master/miscellaneous/gandhi/main.txt', 'corpus/hindi_text_ltrc-master/miscellaneous/skgt/main.txt', 'corpus/hindi_text_ltrc-master/miscellaneous/kamayani/main.txt', 'corpus/hindi_text_ltrc-master/miscellaneous/vaktavya/main.txt', 'corpus/hindi_text_ltrc-master/miscellaneous/seva/main.txt', 'corpus/hindi_text_ltrc-master/cakra/main.txt', 'corpus/hindi_text_ltrc-master/tulasidaas/KV/main.txt', 'corpus/hindi_text_ltrc-master/tulasidaas/Raamacharita_maanasa/5/main.txt', 'corpus/hindi_text_ltrc-master/tulasidaas/Raamacharita_maanasa/6/main.txt', 'corpus/hindi_text_ltrc-master/tulasidaas/Raamacharita_maanasa/1/main.txt', 'corpus/hindi_text_ltrc-master/tulasidaas/Raamacharita_maanasa/2/main.txt', 'corpus/hindi_text_ltrc-master/tulasidaas/Raamacharita_maanasa/3/main.

In [41]:
text_int,idx2char

(array([34, 33, 33, ..., 27, 48, 38]),
 array([' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', ':', '=', '?',
        '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'y', 'z', '|',
        'Ê', 'Ù', 'Ý', 'è', 'é', 'ê', 'í', 'ñ', 'û', 'ā', 'ġ', 'ī', 'ś',
        'ū', '̐', '़', 'ॅ', 'ॉ', 'फ़', 'ḍ', 'ḥ', 'ḷ', 'ḻ', 'ṃ', 'ṅ', 'ṇ',
        'ṛ', 'ṝ', 'ṣ', 'ṭ', '\u200c', '\u200d', '–', '’'], dtype='<U1'))

(array([34, 33, 33, ..., 27, 48, 38]),
 array([' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', ':', '=', '?',
        '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'y', 'z', '|',
        'Ê', 'Ù', 'Ý', 'è', 'é', 'ê', 'í', 'ñ', 'û', 'ā', 'ġ', 'ī', 'ś',
        'ū', '̐', '़', 'ॅ', 'ॉ', 'फ़', 'ḍ', 'ḥ', 'ḷ', 'ḻ', 'ṃ', 'ṅ', 'ṇ',
        'ṛ', 'ṝ', 'ṣ', 'ṭ', '\u200c', '\u200d', '–', '’'], dtype='<U1'))

In [11]:
print((text_int))
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(full_text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

[34 33 33 ... 27 48 38]
u
t
t
a
m


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

uttama jāti hai bāhmanī / rahīma uttama jāti hai bāhmanī, dekhata citta lubhāya| parama pāpa pala meṃ
'uttama jāti hai bāhmanī / rahīma uttama jāti hai bāhmanī, dekhata citta lubhāya| parama pāpa pala meṃ'
 harata, parasata vāke pāya rūparaṃga ratirāja meṃ, chatarānī itarāna| mānau racī biraṃci paci, kusum
' harata, parasata vāke pāya rūparaṃga ratirāja meṃ, chatarānī itarāna| mānau racī biraṃci paci, kusum'
a kanaka meṃ sāna baniyāini bani āikai, baiṭhi rūpa kī hāṭa| pema peka tana herikai, garuvai ṭārati b
'a kanaka meṃ sāna baniyāini bani āikai, baiṭhi rūpa kī hāṭa| pema peka tana herikai, garuvai ṭārati b'
āṭa garaba tarājū karati cakha, bhauṃha mori musakāti| ḍām̐fī mārati biraha kī, cita ciṃtā ghaṭi jāti
'āṭa garaba tarājū karati cakha, bhauṃha mori musakāti| ḍām̐fī mārati biraha kī, cita ciṃtā ghaṭi jāti'
 kamala-dala nainani kī unamāni / rahīma kamala-dala nainani kī unamāni| bisarata nāhiṃ sakhī mo mana
' kamala-dala nainani kī unamāni / rahīma kamala-dala nainani kī unamāni| 

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'uttama jāti hai bāhmanī / rahīma uttama jāti hai bāhmanī, dekhata citta lubhāya| parama pāpa pala me'
Target data: 'ttama jāti hai bāhmanī / rahīma uttama jāti hai bāhmanī, dekhata citta lubhāya| parama pāpa pala meṃ'


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(character_set)



In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(character_set),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 73) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           18688     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 73)            74825     
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [53]:
sampled_indices

array([21,  3, 62, 10, 30, 25, 54, 19, 11, 18, 44, 67, 38, 60,  4, 58,  4,
       72, 39, 62, 55, 49, 36, 27, 11, 48, 56, 59, 46, 42,  0,  3,  8, 61,
       29, 10, 65,  3,  2, 70, 69,  8, 11, 46, 16, 52, 61, 47, 55, 26, 32,
       36, 24, 72, 58, 24, 27, 12,  7, 58, 11, 72, 33, 19, 35, 63, 71, 17,
       64, 39, 47, 69, 11, 39, 30, 35, 23, 62,  7, 70, 60,  4,  6, 68, 17,
       46, 11, 45, 32, 54,  3, 11, 41, 40, 42, 49,  2, 47, 14,  4])

array([42, 29, 31, 37, 66,  9, 49, 65, 11, 44, 60, 31, 60,  0, 72,  6, 12,
        1,  9, 18, 55, 27, 70,  2, 49, 43, 71, 17, 19, 41, 59,  4, 66, 40,
        9, 58, 50, 20, 45, 48,  9, 60, 25, 69, 28, 18, 34, 56, 36, 65,  9,
       50, 45, 29, 21,  3, 41, 65,  9, 59, 69,  1, 53,  9, 25, 52, 67, 65,
       29, 70, 61, 15, 41, 20, 47, 67, 33,  6, 21, 10, 29, 69, 67, 65,  1,
       48, 35, 62, 66,  8, 42,  4, 47, 34, 49,  6, 28, 17, 31,  0])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()

print("Next Char Predictions: \n")
temp = ''.join(idx2char[sampled_indices])
new_text = sanscript.transliterate(temp, sanscript.DEVANAGARI, sanscript.IAST)
print(new_text)

Input: 
 'pīche r pīche ātī thī . kabhī sāmane ākara rāstā roka letī aura kahatīr merī yaha durgati tumane kī '

Next Char Predictions: 

=èsciébṭñ‌ê‍ṅû|ॅÙ=v|b=ī–फ़ûûś,stṣġ’Ýṅi:síḷ:iūÝdÊ(Ùṝūṅॉ़lpi‍`gṝ"gḻmirna)yś̐ke||‍dफ़ciṣṛ!lfफ़ā,ūl`:b`फ़i=ॅ


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 73)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.2920275


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
model.save("model.h5")

Epoch 1/10
 35/455 [=>............................] - ETA: 30:34 - loss: 3.3493

In [0]:
def load_data():
    """ Loads the data from the corpus. """


    # Getting character set.
    print("Getting character set...")
    global full_text
    full_text = " ".join(file_contents)
    global character_set
    character_set = get_character_set(full_text)
    print("Character set:", character_set, len(character_set))

    # Process the data.
    data_input = []
    data_output = []
    current_size = 0
    print("Generating data set...")
    bar = progressbar.ProgressBar(max_value=data_set_size)
    while current_size < data_set_size:
        random_file_content = random.choice(file_contents)

        random_string = random_substring_of_length(random_file_content, input_length + output_length)

        random_string_encoded = encode_string(random_string)

        input_sequence = random_string_encoded[:input_length]
        output_sequence = random_string_encoded[input_length:]

        data_input.append(input_sequence)
        data_output.append(output_sequence)

        current_size += 1
        bar.update(current_size)
    bar.finish()

    # Done.
    train_input = np.array(data_input)
    train_output = np.array(data_output)
    return (train_input, train_output)